# Tutorial 2: more advanced optical systems and ray-tracing options.
In the previous tutorial we built a simple optical system consisting of a paraboloid. We then performed a ray-trace from the initial frame of rays to the paraboloid surface, and from the paraboloid we found the focus of the frame by calling the s.findRTfocus method. We did all of this using the tubular ray-trace frame input.

In this tutorial, we will introduce Gaussian ray-trace frames. We will then create an optical setup that is slightly more advanced, where we try to generate a collimated beam from a Gaussian ray-trace beam.   

In [ ]:
%matplotlib notebook

import numpy as np

from src.PyPO.System import System

s = System()

In [ ]:
RTpar = {
        "name"      : "start",
        "nRays"     : 5000,
        "lam"       : 0.01,
        "n"         : 1,
        "seed"      : 0,
        "x0"        : 0.05,
        "y0"        : 0.05
        }

s.createGRTFrame(RTpar)
#s.rotateGrids("start", np.array([0, 10, 0]), obj="frame")
s.translateGrids("start", np.array([0, 0, 100]), obj="frame")
s.plotRTframe("start", project='xy')

We start by making an ellipsoidal reflector. We place the semi-major axis along the z-axis by setting 'orient' to 'z'. 

In [ ]:
ellipse = {
            "name"      : "ellipsoid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "flip"      : True,
            "focus_1"   : np.array([0, 0, 100]),
            "focus_2"   : np.array([0, 0, -100]),
            "orient"    : "z",
            "ecc"       : 0.5,
            "lims_u"    : np.array([0, 30]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([801, 801])
            }

s.addEllipse(ellipse)

plane_t = {
            "name"      : "plane_t",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 10]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([801, 801])
            }

s.addPlane(plane_t)
s.rotateGrids("plane_t", np.array([45, 0, 0]))

parabola = {
            "name"      : "paraboloid",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "focus_1"   : np.array([0, -100, 0]),
            "vertex"    : np.array([-10, -100, 0]),
            "lims_u"    : np.array([0, 5]),
            "lims_v"    : np.array([0, 360]),
            "gcenter"   : np.array([0,-20]),
            "gridsize"  : np.array([801, 801])
            }

s.addParabola(parabola)

plane_out = {
            "name"      : "plane_out",
            "gmode"     : "xy",
            "lims_x"    : np.array([-10, 10]),
            "lims_y"    : np.array([-10, 10]),
            "gridsize"  : np.array([801, 801])
            }

s.addPlane(plane_out)
s.rotateGrids("plane_out", np.array([0, -90, 0]))
s.translateGrids("plane_out", np.array([50, -120, 0]))

s.plotSystem()

In [ ]:
start_ell_RT = {
            "fr_in"     : "start",
            "t_name"    : "ellipsoid",
            "fr_out"    : "fr_ell",
            "device"    : "CPU",
            "tol"       : 1e-6
            }

s.runRayTracer(start_ell_RT)

ell_plane_t_RT = {
            "fr_in"     : "fr_ell",
            "t_name"    : "plane_t",
            "fr_out"    : "fr_plane_t",
            "device"    : "CPU",
            "tol"       : 1e-6
            }

s.runRayTracer(ell_plane_t_RT)

plane_t_par_RT = {
            "fr_in"     : "fr_plane_t",
            "t_name"    : "paraboloid",
            "fr_out"    : "fr_par",
            "device"    : "CPU",
            "t0"        : 120,
            "tol"       : 1e-6
            }

s.runRayTracer(plane_t_par_RT)

par_plane_out_RT = {
            "fr_in"     : "fr_par",
            "t_name"    : "plane_out",
            "fr_out"    : "fr_out",
            "device"    : "CPU",
            "tol"       : 1e-6
            }

s.runRayTracer(par_plane_out_RT)
s.findRTfocus("fr_plane_t")
s.plotSystem(RTframes=["start", "fr_ell", "fr_plane_t", "fr_par", "fr_out"])
s.plotRTframe("fr_out", project='yz')

#s.plotSystem(select=["paraboloid", "plane_out"], RTframes=["fr_par", "fr_out"])
#s.plotRTframe("fr_out", project='yz')